In [35]:
import pandas as pd
import numpy as np
# import matplotlib as mpl
# import matplotlib.pyplot as plt
from datetime import date, timedelta
# from matplotlib.pyplot import cm
from scipy.stats import pearsonr,spearmanr
import sys
import json
# from minepy import MINE
# from minepy import pstats, cstats
import time
import pickle 
from geopy import distance

In [242]:
country_name = 'RUS'
# country_name = 'GBR'
# country_name = 'TUR'
# country_name = 'PAK'
# country_name = 'IND'
# country_name = 'EGY'
# country_name = 'THA'

if country_name == 'IND':
    n_city = 15
else:
    n_city = 10
n_cities = n_city + 1

# if len(country_name) <= 3:
event_file = '~/data/icews/events.sent.new.1991.201703.country/icews_events_{}.json'.format(country_name)
df = pd.read_json(event_file,lines=True)
print(len(df),'all')
df = df.loc[df['Event Date'] > '2010-01-01']
print(len(df),'>=2010')


df2 = df.drop_duplicates(subset=['Country', 'CAMEO Code', 'Event Date',
         'Story ID',  'Sentence Number' ])
print(len(df2),'remove dup')


cities = df2['City'].value_counts()[:n_cities].index.tolist()
cities = np.array([v for v in cities if v != ''])
print(cities)

# with open('../data/{}/cities.txt'.format(country_name),'w') as f:
#     for c in cities:
#         f.write(c+'\n')
    
# geo of locs
geo_list = []
for c in cities:
    geo = df2.loc[df2['City']==c][['Latitude','Longitude']].values[0]
    geo_list.append(tuple(geo))
    
    
    
print(geo_list)
loc_distance = []
target_index = 0
target_geo = geo_list[target_index]
for i in range(len(geo_list)):
    if i == target_index:
        loc_distance.append(0)
    else:
#         dis = distance.distance(target_geo, geo_list[i]).miles
#         print(dis)
        dis = distance.distance(target_geo, geo_list[i]).km
#         print(dis)
#         print()
        loc_distance.append(dis)
print(loc_distance)

924326 all
259374 >=2010
206146 remove dup
['Moscow' 'Moscow Kremlin' 'St. Petersburg' 'Sochi' 'Vladivostok'
 'Makhachkala' 'Groznyy' 'Volgograd' 'Yekaterinburg' 'Smolensk']
[(55.7522, 37.6156), (55.7517, 37.6178), (59.9386, 30.3141), (43.5992, 39.7257), (43.1056, 131.874), (42.9764, 47.5024), (43.312, 45.6889), (48.7194, 44.5018), (56.8519, 60.6122), (54.7818, 32.0401)]
[0, 0.14893690163335738, 636.0188745172288, 1360.069604481923, 6434.465162660547, 1588.3915808008728, 1499.5463879038334, 912.065033972418, 1422.085230230153, 370.4055080629688]


In [243]:

loc_distance = np.array(loc_distance)
with open('../data/{}/geo_raw.pkl'.format(country_name),'wb') as f:
    pickle.dump(loc_distance,f)
    

In [262]:
 
# np.tanh(loc_distance)
print(loc_distance.mean(),loc_distance.std() )
max_ = loc_distance.max() + 100
n = loc_distance/(max_)

1422.319631953158 1766.0616180397078


In [269]:
(1-n)**0.5,(1-n)**5,

(array([1.        , 0.9999886 , 0.95008791, 0.88986636, 0.1237072 ,
        0.87001204, 0.87779118, 0.92758956, 0.88451772, 0.9712441 ]),
 array([1.00000000e+00, 9.99886043e-01, 5.99291214e-01, 3.11349299e-01,
        8.39362892e-10, 2.48457797e-01, 2.71588958e-01, 4.71583405e-01,
        2.93133445e-01, 7.46936927e-01]))

In [234]:
loc_distance_norm = np.array(loc_distance_norm)
print(loc_distance_norm)
1/(1-loc_distance_norm-1e-6)

[0.000e+00 5.144e-01 9.000e-04 5.614e-01 7.846e-01 5.848e-01 1.862e-01
 1.061e-01 7.776e-01 3.818e-01 2.275e-01 1.000e+00 2.900e-01 5.253e-01
 4.990e-02]


array([ 1.00000100e+00,  2.05931231e+00,  1.00090181e+00,  2.27998696e+00,
        4.64254709e+00,  2.40848364e+00,  1.22880466e+00,  1.11869462e+00,
        4.49642310e+00,  1.61760210e+00,  1.29450006e+00, -1.00000000e+06,
        1.40845269e+00,  2.10659808e+00,  1.05252190e+00])

In [174]:
total_dis = sum(loc_distance)
print(total_dis)
print(loc_distance)
np.array(loc_distance)/total_dis
tmp = [round(v/total_dis,3) for v in loc_distance]
tmp = np.array(tmp)
tmp

13363.415087579013
[0, 1147.4905310365705, 2.045005838947556, 1252.4406341724418, 1750.318140834992, 1304.5872312341291, 415.4522901281679, 236.6270992829022, 1734.7242195462695, 851.6418759528574, 507.4275371060643, 2230.7439898998446, 646.9022033377101, 1171.799668924483, 111.2146602836332]


array([0.   , 0.086, 0.   , 0.094, 0.131, 0.098, 0.031, 0.018, 0.13 ,
       0.064, 0.038, 0.167, 0.048, 0.088, 0.008])

In [232]:
# del distance
loc_distance_norm = []
max_dis = max(loc_distance)
for d in loc_distance:
    if d > 0:
        loc_distance_norm.append(round(d/max_dis,4))
    else:
        loc_distance_norm.append(0)
        
loc_distance_norm

[0,
 0.5144,
 0.0009,
 0.5614,
 0.7846,
 0.5848,
 0.1862,
 0.1061,
 0.7776,
 0.3818,
 0.2275,
 1.0,
 0.29,
 0.5253,
 0.0499]

In [226]:
loc_distance_norm = np.array(loc_distance_norm)
t = [round(v,3) for v in 1/(1-(loc_distance_norm)+1e-6)]
t = np.array(t)
t

array([1.000e+00, 2.059e+00, 1.001e+00, 2.280e+00, 4.643e+00, 2.408e+00,
       1.229e+00, 1.119e+00, 4.496e+00, 1.618e+00, 1.294e+00, 1.000e+06,
       1.408e+00, 2.107e+00, 1.053e+00])

In [227]:
import math

# def sigmoid(x):
#     return 1 / (1 + math.exp(-x))
def sigmoid_array(x):                                        
    return 1 / (1 + np.exp(-x))

# sigmoid_array(t)
np.tanh(t)
np.log(t)

array([0.00000000e+00, 7.22220428e-01, 9.99500333e-04, 8.24175443e-01,
       1.53536071e+00, 8.78796527e-01, 2.06200831e-01, 1.12435429e-01,
       1.50318811e+00, 4.81190819e-01, 2.57738196e-01, 1.38155106e+01,
       3.42170258e-01, 7.45265135e-01, 5.16432332e-02])

In [132]:
with open('../data/{}/geo_dis.pkl'.format(country_name),'wb') as f:
    pickle.dump(loc_distance_norm,f)
    

In [133]:
with open('../data/{}/geo_dis.pkl'.format(country_name),'rb') as f:
    data = pickle.load(f)
data

array([0.    , 0.1046, 0.2208, 0.9401, 0.6295, 0.8642, 0.5564, 1.    ,
       0.3031, 0.639 ])

In [228]:
1/1e-12

1000000000000.0

In [140]:
path = '~/data/ACLED/country-jul23/2017-01-01-2021-07-23-Afghanistan.csv'
path = '~/data/ACLED/country-jul23/2018-01-01-2021-07-23-Brazil.csv'
path = '~/data/ACLED/country-jul23/2015-01-01-2021-07-23-Yemen.csv'
# path = '~/data/ACLED/country-jul23/2018-01-01-2021-07-23-Russia.csv'
# path = '~/data/ACLED/country-jul23/2016-01-01-2021-07-23-Turkey.csv'
# path = '~/data/ACLED/country-jul23/2020-01-01-2021-07-23-United_States.csv'
# path = '~/data/ACLED/country-jul23/2017-01-01-2021-07-23-Syria.csv'

country_name = path.split('-')[7][:-4]
print(country_name,'country_name')
 

df = pd.read_csv(path,sep=';')
df = df.drop_duplicates(subset=['data_id'], keep='first')
df['event_date'] = pd.to_datetime(df['event_date']) 
df.columns

Yemen country_name


Index(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty', 'event_date',
       'year', 'time_precision', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'region', 'country', 'admin1', 'admin2', 'admin3',
       'location', 'latitude', 'longitude', 'geo_precision', 'source',
       'source_scale', 'notes', 'fatalities', 'timestamp', 'iso3'],
      dtype='object')

In [141]:
n_cities = 15
cities = df['admin1'].value_counts()[:n_cities].index.tolist()
print(cities)
if  country_name == 'Yemen':
    print(cities[7])
    new_cities = np.concatenate((cities[7:8],cities[:7],cities[8:]))
    print(new_cities)
    cities = new_cities
    
# with open('../data/{}/cities.txt'.format(country_name),'w') as f:
#     for c in cities:
#         f.write(c+'\n')

['Al Hudaydah', 'Sadah', 'Taizz', 'Marib', 'Al Jawf', 'Sanaa', 'Hajjah', 'Al Bayda', 'Ad Dali', 'Aden', 'Amanat al Asimah', 'Abyan', 'Shabwah', 'Lahij', 'Ibb']
Al Bayda
['Al Bayda' 'Al Hudaydah' 'Sadah' 'Taizz' 'Marib' 'Al Jawf' 'Sanaa'
 'Hajjah' 'Ad Dali' 'Aden' 'Amanat al Asimah' 'Abyan' 'Shabwah' 'Lahij'
 'Ibb']


In [130]:
geo_list = []
for c in cities:
    print(c)
    geo = df.loc[df['admin1']==c][['latitude', 'longitude']].values
#     print(geo.mean(0),geo.std(0))
    geo = geo.mean(0)
    geo_list.append(tuple(geo))
    print(geo)
    
print(geo_list)
loc_distance = []
target_index = 0
target_geo = geo_list[target_index]
for i in range(len(geo_list)):
    if i == target_index:
        loc_distance.append(0)
    else:
#         dis = distance.distance(target_geo, geo_list[i]).miles
#         print(dis)
        dis = distance.distance(target_geo, geo_list[i]).km
#         print(dis)
#         print()
        loc_distance.append(dis)
print(loc_distance)

Idleb
[35.71163978 36.62262623]
Hama
[35.35219238 36.67303294]
Aleppo
[36.33187864 37.17182317]
Deir ez Zor
[34.99716551 40.49717196]
Rural Damascus
[33.53982176 36.41216   ]
Dara
[32.74884597 36.1280145 ]
Ar Raqqa
[36.07587439 38.94605299]
Al Hasakeh
[36.60915252 40.74388588]
Homs
[34.78284711 37.21700364]
Damascus
[33.51611618 36.30231132]
[(35.711639781021894, 36.62262622596895), (35.35219237653929, 36.67303293766662), (36.331878635250035, 37.171823173367926), (34.99716550670884, 40.497171964655834), (33.539821760167385, 36.41215999738459), (32.7488459725284, 36.12801449889775), (36.07587439458689, 38.94605299145299), (36.609152519178934, 40.743885880157585), (34.78284711005543, 37.21700364212193), (33.51611618257261, 36.30231132187314)]
[0, 40.141977857573586, 84.77591279428506, 360.95188009713723, 241.69957922880553, 331.7954908518547, 213.62127690788137, 383.9375562330946, 116.38072468093328, 245.32210457664243]


In [131]:
loc_distance_norm = []
max_dis = max(loc_distance)
for d in loc_distance:
    if d > 0:
        loc_distance_norm.append(round(d/max_dis,4))
    else:
        loc_distance_norm.append(0)
        
loc_distance_norm = np.array(loc_distance_norm)

/home/sdeng/anaconda2/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


(array([        inf, 10.56022945,  5.52898551,  2.06371663,  2.58856235,
         2.15713955,  2.79726815,  2.        ,  4.29924117,  2.56494523]),
 array([0.    , 0.1046, 0.2208, 0.9401, 0.6295, 0.8642, 0.5564, 1.    ,
        0.3031, 0.639 ]))

In [105]:
df.loc[(df['admin1']==df['admin2']) & (df['admin3']==df['admin2'])][['admin1','admin2','admin3','latitude', 'longitude']]

,admin1,admin2,admin3,latitude,longitude
1,Dara,Dara,Dara,32.6241,36.1049
2,Al Hasakeh,Al Hasakeh,Al Hasakeh,36.5054,40.7429
3,Ar Raqqa,Ar Raqqa,Ar Raqqa,35.9428,39.0519
14,Ar Raqqa,Ar Raqqa,Ar Raqqa,35.9360,38.9826
23,Idleb,Idleb,Idleb,35.8726,36.6313
...,...,...,...,...,...
90883,Deir ez Zor,Deir ez Zor,Deir ez Zor,35.3205,40.1738
90887,Deir ez Zor,Deir ez Zor,Deir ez Zor,35.4251,40.1252
90903,Deir ez Zor,Deir ez Zor,Deir ez Zor,35.2856,40.1770
90914,Deir ez Zor,Deir ez Zor,Deir ez Zor,35.3225,40.1556


In [143]:
temp = open('../data/{}/cities.txt'.format(country_name)).read().splitlines()
temp

['Al Bayda',
 'Al Hudaydah',
 'Sadah',
 'Taizz',
 'Marib',
 'Al Jawf',
 'Sanaa',
 'Hajjah',
 'Ad Dali',
 'Aden',
 'Amanat al Asimah',
 'Abyan',
 'Shabwah',
 'Lahij',
 'Ibb']

In [164]:
print(loc_distance_norm)

o = (1-loc_distance_norm)**1
print(o,'o')
1/(o+1e-6)

[0.     0.1046 0.2208 0.9401 0.6295 0.8642 0.5564 1.     0.3031 0.639 ]
[1.     0.8954 0.7792 0.0599 0.3705 0.1358 0.4436 0.     0.6969 0.361 ] o


array([9.99999000e-01, 1.11681805e+00, 1.28336591e+00, 1.66942121e+01,
       2.69904805e+00, 7.36371603e+00, 2.25427806e+00, 1.00000000e+06,
       1.43492404e+00, 2.77007543e+00])